In [233]:
import pandas as pd
import numpy as np

In [234]:
store_inventory = pd.read_csv('Supplemental Order Store Data.csv')
forecast = pd.read_csv('Supplemental Order Forecast Data.csv')
dc_inventory = pd.read_csv('DC On Hand.csv')

## Calculating Total Pipeline

In [235]:
store_inventory['store_in_transit_quantity_this_year'] = pd.to_numeric(store_inventory['store_in_transit_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_in_warehouse_quantity_this_year'] = pd.to_numeric(store_inventory['store_in_warehouse_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_on_hand_quantity_this_year'] = pd.to_numeric(store_inventory['store_on_hand_quantity_this_year'], errors='coerce').fillna(0)
store_inventory['store_on_order_quantity_this_year'] = pd.to_numeric(store_inventory['store_on_order_quantity_this_year'], errors='coerce').fillna(0)

# Now create the 'total_pipeline' column
store_inventory['total_pipeline'] = (
    store_inventory['store_in_transit_quantity_this_year'] + 
    store_inventory['store_in_warehouse_quantity_this_year'] + 
    store_inventory['store_on_hand_quantity_this_year'] + 
    store_inventory['store_on_order_quantity_this_year']
)

# converting the whse on hand column to numeric
dc_inventory['on_hand_warehouse_inventory_in_units_this_year'] = dc_inventory['on_hand_warehouse_inventory_in_units_this_year'].str.replace(',', '')
dc_inventory['on_hand_warehouse_inventory_in_units_this_year'] = pd.to_numeric(dc_inventory['on_hand_warehouse_inventory_in_units_this_year'], errors='coerce').fillna(0)

## Calculating Needed Quantity

In [236]:
# Initialize an empty list to store the data
data = []

# getting unique list of items
items = store_inventory['vendor_stock_id'].unique()

for item in items:
    all_links_item_number = store_inventory[store_inventory['vendor_stock_id'] == item]['all_links_item_number'].values[0]

    # filtering the list of stores once we select the item
    inventory_filtered_item = store_inventory[store_inventory['vendor_stock_id'] == item]

    # sorting the stores based on the total pipeline ascending
    inventory_filtered_item = inventory_filtered_item.sort_values(by='total_pipeline', ascending=True)

    # getting the unique list of stores
    stores = inventory_filtered_item['store_number'].unique()

    total_shipped = 0
    for store in stores:
        # getting the total pipeline and vendor pack quantity for the selected variables
        inventory_filtered = inventory_filtered_item[inventory_filtered_item['store_number'] == store]
        total_pipeline = inventory_filtered['total_pipeline'].values[0]
        vendor_pack_quantity = inventory_filtered['vendor_pack_quantity'].values[0]
        max_shelf_quantity = inventory_filtered['max_shelf_quantity_this_year'].values[0]
        store_on_hand_quantity = inventory_filtered['store_on_hand_quantity_this_year'].values[0]
        dc = inventory_filtered['distribution_center_number'].values[0]
        idc = inventory_filtered['storage_distribution_center_number'].values[0]

        # getting the forecasted quantity for the selected variables
        forecast_filtered = forecast[(forecast['vendor_stock_id'] == item) & (forecast['store_nbr'] == store)]
        if not forecast_filtered.empty:
            forecasted_quantity = forecast_filtered['final_fcst_each_qty'].values[0]
        else:
            forecasted_quantity = 0  # Handle the case when no forecast is available

        # calculating the needed quantity for the item and store
        needed_quantity = total_pipeline - forecasted_quantity
        #max_shelf_minus_pipeline = max_shelf_quantity - total_pipeline
        if needed_quantity > 0:
            needed_quantity = 0
        else:
            needed_quantity = abs(needed_quantity)
            whse_packs_needed = np.ceil(needed_quantity / vendor_pack_quantity)
            needed_quantity = whse_packs_needed * vendor_pack_quantity
            #needed_quantity = needed_quantity - max_shelf_quantity
            if needed_quantity > max_shelf_quantity:
                needed_quantity = np.ceil(max_shelf_quantity / vendor_pack_quantity)
            else:
                #needed_quantity = needed_quantity
                needed_quantity = np.ceil(needed_quantity / vendor_pack_quantity)

        # Append the result to the list, including all_links_item_number
        data.append({
            'Item': item,
            'Store': store,
            'IDC': idc,
            'DC': dc,
            'Needed Quantity': needed_quantity,
            'All Links Item Number': all_links_item_number
        })
# Convert the list of dictionaries into a DataFrame
needed_quantity_df = pd.DataFrame(data)

# remove zeros
needed_quantity_df = needed_quantity_df[needed_quantity_df['Needed Quantity'] > 0]

## Sorting by Item and by Needed Quantity

This ensure that we send the inventory to the stores that need the most first

In [237]:
# sort the dataframe based on item then needed quantity
needed_quantity_df = needed_quantity_df.sort_values(by=['All Links Item Number','Needed Quantity'], ascending=[True,False])

# SUM THE NEEDED QUANTITY FOR EACH ITEM
total_quantity_needed = needed_quantity_df.groupby(['All Links Item Number']).sum()
total_quantity_needed

,Item,Store,IDC,DC,Needed Quantity
All Links Item Number,,,,,
585320691,646503,280809,853872.0,741947.0,122.0
597319350,343525,157238,460955.0,400579.0,65.0
659879048,1352448,1381140,4092263.0,3716663.0,903.0
663269028,1605786,1565045,4813944.0,4351244.0,836.0
663269050,1467075,1494468,4215358.0,3873857.0,952.0
663398046,3261013,3366958,9724181.0,8625955.0,2060.0


## Calculating the Fullfilled Amount

In [239]:
# Function to handle the row-by-row subtraction
def subtract_needed_quantity_by_item(needed_quantity_df, dc_inventory):
    # Loop through each unique item in the needed_quantity_df
    for item in needed_quantity_df['Item'].unique():

        # List of IDC numbers
        idc_numbers = [6060, 6061, 6088, 7042, 7067, 7078, 8980]

        # Dictionary comprehension to calculate the sum for each IDC
        idc_oh = {idc: dc_inventory.loc[
                    (dc_inventory['distribution_center_number'] == idc) & 
                    (dc_inventory['vendor_stock_id'] == str(item)),
                    'on_hand_warehouse_inventory_in_units_this_year'
                ].sum() 
                for idc in idc_numbers}

        # List of DC numbers
        dc_numbers = [6021, 6026, 6031, 6037, 7026, 7033, 6010, 6020, 6054, 7035, 7038, 6023, 6027, 6030, 6038, 
                    6080, 7034, 6012, 6016, 6019, 6035, 6036, 6068, 6094, 7036, 6006, 6011, 6017, 6018, 6048, 
                    6066, 6069, 6009, 6025, 6043, 6092, 7039, 6024, 6039, 6040, 6070, 7045]

        # Dictionary comprehension to calculate the sum for each DC
        dc_oh = {dc: dc_inventory.loc[
                    (dc_inventory['distribution_center_number'] == dc) & 
                    (dc_inventory['vendor_stock_id'] == str(item)),
                    'on_hand_warehouse_inventory_in_units_this_year'
                ].sum() 
                for dc in dc_numbers}  

        for idx, row in needed_quantity_df[needed_quantity_df['Item'] == item].iterrows():
            dc = row['DC']
            idc = row['IDC']
            needed_qty = row['Needed Quantity']
            # Check if we have enough in the total available quantity to subtract the needed quantity
            if dc_oh[dc] >= needed_qty:
                # If enough, subtract the needed quantity from available_quantity
                dc_oh[dc] -= needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity DC'] = needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity IDC'] = 0
            elif idc_oh[idc] >= needed_qty:
                # If enough, subtract the needed quantity from available_quantity
                idc_oh[idc] -= needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity IDC'] = needed_qty
                needed_quantity_df.at[idx, 'Fulfilled Quantity DC'] = 0
            else:
                # If not enough set fulfill to zero
                needed_quantity_df.at[idx, 'Fulfilled Quantity DC'] = 0
                needed_quantity_df.at[idx, 'Fulfilled Quantity IDC'] = 0

    return needed_quantity_df

# Apply the function to subtract from available inventory based on needed quantity
fulfilled_df = subtract_needed_quantity_by_item(needed_quantity_df, dc_inventory)

In [240]:
fulfilled_df

,Item,Store,IDC,DC,Needed Quantity,All Links Item Number,Fulfilled Quantity IDC,Fulfilled Quantity DC
7730,5343,154,8980.0,6039.0,1.0,585320691,1.0,0.0
7600,5343,1500,7078.0,6025.0,1.0,585320691,0.0,0.0
7601,5343,1534,7078.0,6025.0,1.0,585320691,0.0,0.0
7602,5343,1542,7078.0,6043.0,1.0,585320691,0.0,0.0
7603,5343,1546,7078.0,6009.0,1.0,585320691,0.0,0.0
...,...,...,...,...,...,...,...,...
15353,2341,1915,6060.0,7033.0,1.0,663398046,1.0,0.0
15412,2341,1555,6060.0,7033.0,1.0,663398046,1.0,0.0
15438,2341,1747,6060.0,7033.0,1.0,663398046,1.0,0.0
15456,2341,3350,6060.0,7033.0,1.0,663398046,1.0,0.0


## Finalizing the Dataset

In [241]:
# Fixing the slice warning using .loc[]
sto_single = sto_single.loc[:, ~sto_single.columns.isin(['Fulfilled Quantity IDC', 'Needed Quantity', 'IDC'])]

sto_multi = sto_multi.loc[:, ~sto_multi.columns.isin(['Fulfilled Quantity DC', 'Needed Quantity', 'DC'])]

## Overview of Needed and Fullfilled Quantity

In [247]:
# summing total of fulled quantity for each item
total_fulfilled_quantity = fulfilled_df.groupby(['Item']).sum()
total_fulfilled_quantity['Total_fulfilled'] = total_fulfilled_quantity['Fulfilled Quantity DC'] + total_fulfilled_quantity['Fulfilled Quantity IDC']
total_fulfilled_quantity = total_fulfilled_quantity.drop(columns=['Store','IDC','DC','All Links Item Number','Fulfilled Quantity IDC','Fulfilled Quantity DC'])
total_fulfilled_quantity

,Needed Quantity,Total_fulfilled
Item,,
2249,836.0,177.0
2304,903.0,474.0
2325,952.0,211.0
2341,2060.0,328.0
5285,65.0,2.0
5343,122.0,8.0


## Exporting the Data as a CSV

In [242]:
sto_multi.to_csv('sto_multi.csv', index=False)
sto_single.to_csv('sto_single.csv', index=False)